In [1]:
import zipfile
import os, shutil
import collections
from pathlib import Path
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.regularizers import l2
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, concatenate
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle
import pylab as pl
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import itertools
import math


# audio lib
import librosa
import librosa.display
from librosa.util import fix_length

import IPython.display as ipd
from PIL import Image

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
output_folder = "/kaggle/working/melspectrogram_tmp/"
os.mkdir(output_folder)

In [4]:
df = pd.read_csv('../input/my-playlist/my-playlist.csv', sep=';', dtype = str)


True_Label = df['Hidden_True_Label']

del df['Hidden_True_Label']
del df['Label1']

df = df.dropna()

df

,ID
0,Fabrizio De Andr - La canzone di Marinella
1,John Coltrane - Naima
2,Nirvana - Come As You Are
3,Queen - Radio Ga Ga
4,Red Hot Chili Peppers - By The Way
5,Imagine Dragons - On Top of the World
6,Green Day - Time of Your Life
7,Oasis - Wonderwall
8,Eagles - Hotel California
9,Red Hot Chili Peppers - Snow


In [5]:
base_net = load_model('../input/effnetb3spectrogram/weights.cnn.hdf5')
layer_name = 'flatten'
feature_extractor = Model(inputs=base_net.input, outputs=base_net.get_layer(layer_name).output)

2022-07-18 15:57:57.252169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 15:57:57.344979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 15:57:57.345829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 15:57:57.347776: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
LSTM = load_model('../input/lstmspectrogram/weights.cnn.hdf5')

# Verifica classificazione su playlist

In [7]:
# cancello la directory temporanea e la ricero
shutil.rmtree(output_folder)
os.mkdir(output_folder)

In [8]:
data = {
    'ID' : [],
    'A-V-' : [],
    'A-V+' : [],
    'A+V-' : [],
    'A+V+' : []
}

directory = '../input/my-playlist/Playlist/'

SAMPLE_RATE = 22050
TRACK_DURATION = 480
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

num_segments = 80
samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)


num_mel = 96
n_fft = 2048
hop_length = 512

num_mel_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

In [9]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [10]:
print("Classificazione mood canzioni in corso...")
print("Legenda:")
print("0: basso Arousal - basso valence ")
print("1: basso Arousal - alto valence ")
print("2: alto Arousal - basso valence ")
print("3: alto Arousal - alto valence ")
print("----------------------------------------------------")

# per ogni file musicale genero uno spectogramma temporaneo
for filename in os.listdir(directory):
    labels = []
    
    filename_no_extension = Path(filename).stem
    data["ID"].append(filename_no_extension)
    
    file_path = os.path.join(directory, filename)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE,  offset=5.0, duration=TRACK_DURATION)

    # genero spectogrammi per ogni 6 secondi di canzone
    for d in range(num_segments):

                    # calculate start and finish sample for current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # extract mel
                    mel = librosa.feature.melspectrogram(y=signal[start:finish], sr=sample_rate, n_mels=num_mel, hop_length=hop_length)
                    mel = mel.T

                    
                    if (len(mel) == num_mel_vectors_per_segment):
                        
                        
                        img_name = str(d) + '.png'

                        


                        fig = plt.figure(figsize=(3, 3), dpi=100)
                        S_dB = librosa.power_to_db(mel.T)
                        img = librosa.display.specshow(S_dB)
                        
                        plt.savefig("{}{}".format(output_folder,img_name))
                        plt.close(fig)

    # per ogni 4 spettrogrammi generati dalla canzone eseguo la predizione e creo una lista con tutte le predizioni riferite ad una canzone
    counter_spectrogram = 0
    pic_array = []
    features_spectrogram = []
    
    dir_spectogram = sorted_alphanumeric(os.listdir(output_folder))
    for i in dir_spectogram:
        
        counter_spectrogram = counter_spectrogram + 1 # incremento il contatore di frame

        # ottengo le features
        file_path = os.path.join(output_folder, i)
        pic = image.load_img(file_path)
        pic = np.array(pic)
        pic_array.append(pic)

            
        if counter_spectrogram == 4:
            pic_array = np.array(pic_array)
            features_spectrogram = feature_extractor.predict(pic_array)
            features_spectrogram = np.reshape(features_spectrogram, (1, 4, -1))
            classification = LSTM.predict(features_spectrogram)
            labels.append(np.argmax(classification))
            
            counter_spectrogram = 0
            pic_array = []
            features_spectrogram = []
    
    
    # creo una lista con le percentuali di mood per ogni canzone

    counter = collections.Counter(labels)
    most_common= [(i, round(counter[i] / len(labels) * 100.0, 1)) for i, count in counter.most_common()]
    
    # trasformo la lista in campi per il dataframe
    i0 = False
    i1 = False
    i2 = False
    i3 = False
    
    for i in range(len(most_common)):
        if most_common[i][0] == 0:
            i0 = True
            data['A-V-'].append(most_common[i][1])
        elif most_common[i][0] == 1:
            i1 = True
            data['A-V+'].append(most_common[i][1])
        elif most_common[i][0] == 2:
            i2 = True
            data['A+V-'].append(most_common[i][1])
        elif most_common[i][0] == 3:
            i3 = True
            data['A+V+'].append(most_common[i][1])
            
    
    if not(i0):
        data['A-V-'].append(0)
    if not(i1):
        data['A-V+'].append(0)
    if not(i2):
        data['A+V-'].append(0)
    if not(i3):
        data['A+V+'].append(0)

            
        
    

    print("File: "+filename)
    print("Predizioni:")
    print(labels)
    print("Mood in percentuale: "+ str(most_common))    
    print("----------------------------------------------------")
    
    # cancello la directory temporanea e la ricero
    shutil.rmtree(output_folder)
    os.mkdir(output_folder)

Classificazione mood canzioni in corso...
Legenda:
0: basso Arousal - basso valence 
1: basso Arousal - alto valence 
2: alto Arousal - basso valence 
3: alto Arousal - alto valence 
----------------------------------------------------


2022-07-18 15:58:18.357489: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-18 15:58:20.121901: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


File: Fabrizio De Andr - La canzone di Marinella.mp3
Predizioni:
[1, 1, 1, 1, 1, 1, 1]
Mood in percentuale: [(1, 100.0)]
----------------------------------------------------
File: John Coltrane - Naima.mp3
Predizioni:
[0, 0, 1, 1, 1, 0, 1, 0, 0, 0]
Mood in percentuale: [(0, 60.0), (1, 40.0)]
----------------------------------------------------
File: Nirvana - Come As You Are.mp3
Predizioni:
[1, 3, 3, 3, 3, 2, 3, 3, 0]
Mood in percentuale: [(3, 66.7), (1, 11.1), (2, 11.1), (0, 11.1)]
----------------------------------------------------
File: Queen - Radio Ga Ga.mp3
Predizioni:
[0, 0, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3]
Mood in percentuale: [(3, 71.4), (0, 14.3), (2, 14.3)]
----------------------------------------------------
File: Red Hot Chili Peppers - By The Way.mp3
Predizioni:
[0, 2, 3, 2, 2, 2, 2, 3]
Mood in percentuale: [(2, 62.5), (3, 25.0), (0, 12.5)]
----------------------------------------------------
File: Imagine Dragons - On Top of the World.mp3
Predizioni:
[1, 3, 2, 3, 3, 

Una volta ottenute le varie inferenze sulle canzoni vengono confrontati i valori reali e quelli ottenuti dal modello.

In [11]:
df = pd.DataFrame(data)

for index,row in df.iterrows():
    if row[1] >= row[2] and row[1] >= row[3] and row[1] >= row[4]:
        df.loc[index,'Max_Overall_Emotion'] = '0'
    elif row[2] >= row[3] and row[2] >= row[4]:
        df.loc[index,'Max_Overall_Emotion'] = '1'
    elif row[3] >= row[4]:
        df.loc[index,'Max_Overall_Emotion'] = '2'
    else:
        df.loc[index,'Max_Overall_Emotion'] = '3'
        
df['True_Emotion'] = True_Label

for index,row in df.iterrows():
    if row[5] == row[6]:
        df.loc[index,'Wrong_Inferences'] = False
    else: # la predizione è corretta se il valore predetto è uguale al valore originale
        if row[(int(row[6])+1)] - row[(int(row[5])+1)] == 0: 
            df.loc[index,'Wrong_Inferences'] = False
        else:
            df.loc[index,'Wrong_Inferences'] = True
        
    df.loc[index, 'diff_abs'] = abs(row[(int(row[6])+1)] - row[(int(row[5])+1)])
        

df.to_csv('finetuned.csv',index=False)

df

,ID,A-V-,A-V+,A+V-,A+V+,Max_Overall_Emotion,True_Emotion,Wrong_Inferences,diff_abs
0,Fabrizio De Andr - La canzone di Marinella,0.0,100.0,0.0,0.0,1,0,True,100.0
1,John Coltrane - Naima,60.0,40.0,0.0,0.0,0,0,False,0.0
2,Nirvana - Come As You Are,11.1,11.1,11.1,66.7,3,2,True,55.6
3,Queen - Radio Ga Ga,14.3,0.0,14.3,71.4,3,3,False,0.0
4,Red Hot Chili Peppers - By The Way,12.5,0.0,62.5,25.0,2,3,True,37.5
5,Imagine Dragons - On Top of the World,0.0,14.3,14.3,71.4,3,3,False,0.0
6,Green Day - Time of Your Life,16.7,50.0,0.0,33.3,1,1,False,0.0
7,Oasis - Wonderwall,11.1,11.1,0.0,77.8,3,1,True,66.7
8,Eagles - Hotel California,31.2,50.0,6.2,12.5,1,3,True,37.5
9,Red Hot Chili Peppers - Snow,18.2,18.2,0.0,63.6,3,1,True,45.4


In [12]:
print("Errori di classificazione:")
for index,row in df.iterrows():
    if row[7]:
        print("Canzone: "+row[0])
        print("Emozione vera: " +row[6])
        print("Emozione trovata: " +row[5])
        print("Valore stimato emozione vera: "+str(row[(int(row[6])+1)]))
        print("Valore stimato emozione trovata: "+str(row[(int(row[5])+1)]))
        print("Differenza in valore assoluto: {:.2f}".format(row[8]))
        print("------------------------------------------------")

Errori di classificazione:
Canzone: Fabrizio De Andr - La canzone di Marinella
Emozione vera: 0
Emozione trovata: 1
Valore stimato emozione vera: 0.0
Valore stimato emozione trovata: 100.0
Differenza in valore assoluto: 100.00
------------------------------------------------
Canzone: Nirvana - Come As You Are
Emozione vera: 2
Emozione trovata: 3
Valore stimato emozione vera: 11.1
Valore stimato emozione trovata: 66.7
Differenza in valore assoluto: 55.60
------------------------------------------------
Canzone: Red Hot Chili Peppers - By The Way
Emozione vera: 3
Emozione trovata: 2
Valore stimato emozione vera: 25.0
Valore stimato emozione trovata: 62.5
Differenza in valore assoluto: 37.50
------------------------------------------------
Canzone: Oasis - Wonderwall
Emozione vera: 1
Emozione trovata: 3
Valore stimato emozione vera: 11.1
Valore stimato emozione trovata: 77.8
Differenza in valore assoluto: 66.70
------------------------------------------------
Canzone: Eagles - Hotel Calif

In [13]:
print("Numero canzoni corrette: " + str(df.Wrong_Inferences.value_counts()[0]))
print("Numero canzoni errate: " + str(df.Wrong_Inferences.value_counts()[1]))
print("Differenza media predizione errate: {:.2f}".format(df[df.Wrong_Inferences == True].diff_abs.mean()))

Numero canzoni corrette: 23
Numero canzoni errate: 27
Differenza media predizione errate: 47.73
